In [1]:
"""
CODE BASED ON:
https://stackoverflow.com/questions/41433730/python-is-there-a-way-to-scrape-the-abstract-text-from-the-articles-within-each
Searching and Scraping PubMed for Journals' Scientific Articles on COVID-19 - 2019-nCoV
Team: minsky.cc - PE
"""

"\nCODE BASED ON:\nhttps://stackoverflow.com/questions/41433730/python-is-there-a-way-to-scrape-the-abstract-text-from-the-articles-within-each\nSearching and Scraping PubMed for Journals' Scientific Articles on COVID-19 - 2019-nCoV\nTeam: minsky.cc - PE\n"

In [6]:
import unicodecsv as csv
from lxml import html
import lxml.html.clean
import requests

In [7]:
csv_out = open('PubMed_COVID19_Results.csv', 'ab')

In [8]:
writer = csv.writer(csv_out, dialect='excel', delimiter=',', encoding='utf-8')

In [9]:
writer.writerow(['Search_Term', 'Result', 'Title', 'URL', 'Abstract'])

39

In [10]:
# Search Strategy: "COVID-19" [Supplementary Concept] OR 2019-nCoV OR COVID-19 OR SARS-CoV-2 OR coronavirus
Search_Term = '%22COVID-19%22+%5BSupplementary+Concept%5D+OR+2019-nCoV+OR+COVID-19+OR+SARS-CoV-2+OR+coronavirus'

In [11]:
# Displaying max. 200 results
Search_URL = 'https://www.ncbi.nlm.nih.gov/pubmed?term=' + Search_Term + '&dispmax=200'

In [12]:
Search_Page = requests.get(Search_URL)

In [13]:
Search_Tree = html.fromstring(Search_Page.content)

In [14]:
# total number of results
Search_Results = Search_Tree.xpath('//h3[@class="result_count left"]/text()')

In [15]:
Num_Results = str([' '.join(str(result).split()) for result in Search_Results])

In [16]:
Num_Results_Val = Num_Results[Num_Results.find('of') + 3:-2]

In [17]:
# Links for results 1-200
title_cleaner = lxml.html.clean.Cleaner(allow_tags=['div', 'p', 'a'], remove_unknown_tags=False)

In [18]:
Title_Tree = title_cleaner.clean_html(Search_Tree)

In [19]:
Pub_Results = Title_Tree.xpath('//div[@class="rprt"]/div[@class="rslt"]/p[@class="title"]/a')

In [20]:
r = 1

In [21]:
# This will take a while, so wait until it finishes
for Pub_Result in Pub_Results:
    Result_Num = str(r) + '/' + str(Num_Results_Val)
    Pub_Title = ' '.join(Pub_Result.text_content().split())
    Rel_URL = Pub_Result.get('href')
    Pub_URL = Rel_URL.replace('/pubmed/', 'https://www.ncbi.nlm.nih.gov/pubmed/')
    Pub_Page = requests.get(Pub_URL)
    Pub_Tree = html.fromstring(Pub_Page.content)
    Abstract = ''.join(Pub_Tree.xpath('//abstracttext/text()'))
    writer.writerow([Search_Term, Result_Num, Pub_Title, Pub_URL, Abstract])
    r += 1

In [22]:
csv_out.close()
exit()